# Python training SQL

Some basic questions:
- In this database, show the customers, products, finally the orders. 
- Display customers that put stuff in their c.rts, who ordered, pending carts.

Operations:
- Display the total number of customers, products and orders.
- In this database, show the number of pending carts. 
- Orders amount (let's change the times, 1 month)
- Find the t.p 5 c.st.m..s, pr.d.cts.
- F.nd the fl.p 5 c.st.m.rs and pr.d.cts.
- Find the av.r.ge t.t.l ord.rs, pr.d.cts pr.ce.
- F.nd the m.d..n ord.r, pr.d.ct pr.ce.
- F.nd qu.rt.l.s
- 
- 

### In this database, show the customers, products, finally the orders. 

In [ ]:
USE `Test_SQL`;

SELECT * FROM customers;
SELECT * FROM products;
SELECT * FROM orders; 

### In this database, display customers who ordered,  also with pending carts.

In [ ]:
USE `Test_SQL`;

SELECT DISTINCT C.customer_id FROM carts as C; -- with carts
SELECT C.customer_id FROM customers as C; -- the total

SELECT DISTINCT C.customer_id, count(O.cart_id) -- every customers, counting the number of times, that ordered
FROM orders as O 
OUTER JOIN carts as C 
ON C.cart_id = O.cart_id
GROUP BY C.customer_id
ORDER BY C.customer_id;

SELECT DISTINCT C.customer_id, count(C.cart_id) -- all carts
FROM carts as C 
GROUP BY C.customer_id
ORDER BY C.customer_id;

SELECT DISTINCT C.customer_id, count(C.cart_id) -- the p.nd.ng c.rts
FROM carts as C 
WHERE C.total = 0
GROUP BY C.customer_id
ORDER BY C.customer_id;

# Calculations

### In this database, display the total number of customers, products and orders.

In [ ]:
USE `Test_SQL`;

SELECT count(C.customer_id) FROM customers as C;
SELECT count(O.order_id) FROM orders as O;
SELECT count(P.product_id) FROM products as P;

### In this database, show the number of pending carts. 

In [ ]:
-- all pending carts
WITH  pending_carts AS 
	(SELECT DISTINCT C.customer_id, count(C.cart_id) as cart_count
	FROM carts as C 
	WHERE C.total = 0
	GROUP BY C.customer_id
	ORDER BY C.customer_id) 
		SELECT sum(cart_count)
        FROM pending_carts;


### In this database, display the total orders amount (let's change the times, 1 month)

In [ ]:
SELECT sum(C.total) 
FROM carts as C, orders as O 
WHERE C.total > 0 AND O.cart_id = C.cart_id;

### In this database, find the top 5 customers (ord.r.d by sp.nd.ng am..nt)

In [ ]:
SELECT C.customer_id, SUM(C.total)
FROM carts as C, orders as O
WHERE O.cart_id = C.cart_id
GROUP BY C.customer_id
ORDER BY -SUM(C.total)
LIMIT 5;

### In this database find the flop 5 customers (ord.r.d by sp.nd.ng am..nt)

In [ ]:
SELECT C.customer_id, SUM(C.total)
FROM carts as C, orders as O
WHERE O.cart_id = C.cart_id
GROUP BY C.customer_id
ORDER BY SUM(C.total) -- delete the "-"
LIMIT 5;

### In this database, find the top 5 products. (ordered m.st, qu.nt.ty)

In [ ]:
# What is the f.st.st meth.d?

SELECT P.product_id, P.product_name, SUM(Joint.quantity)
FROM 
    carts as C, 
    orders as O, 
    cart_products as Joint, 
    products as P
WHERE 
    O.cart_id = C.cart_id 
    AND C.total > 0 
    AND C.cart_id = Joint.cart_id 
    AND Joint.product_id = P.product_id
GROUP BY 
    P.product_name, 
    P.product_id
ORDER BY 
    -SUM(Joint.quantity)
LIMIT 5;

# Tim.ng (ex.cut..n t.me): 0.00198603 s

SELECT P.product_id, P.product_name, SUM(Joint.quantity)
	FROM products as P
	INNER JOIN cart_products as Joint
    ON Joint.product_id = P.product_id
		INNER JOIN carts as C
        ON C.cart_id = Joint.cart_id
			INNER JOIN orders as O
            ON O.cart_id = C.cart_id
				WHERE C.total > 0
                GROUP BY P.product_name, P.product_id
				ORDER BY -SUM(Joint.quantity)
				LIMIT 5;
                
# Tim.ng (ex.cut..n t.me): 0.00216079 s (time +8,79% )


### In this database, find the average total orders, pr.d.cts pr.ce.

In [ ]:
SELECT C.customer_id, AVG(C.total)
FROM carts as C, orders as O
WHERE O.cart_id = C.cart_id AND C.total > 0
GROUP BY C.customer_id
ORDER BY -AVG(C.total);

SELECT AVG(P.price)
FROM products as P

### In this database, find the median order, product price.

In [ ]:
#https://www.sisense.com/blog/medians-in-sql/
set @ct := (select count(1) from carts);
set @row_id := 0;

select avg(truc.total) as median
from (select * from carts order by carts.total) as truc, orders
where (select @row_id := @row_id + 1 as machin) AND truc.total > 0 AND truc.cart_id = orders.cart_id
between @ct/2.0 and @ct/2.0 + 1;

# https://stackoverflow.com/questions/1291152/simple-way-to-calculate-median-with-mysql
SELECT CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(
 GROUP_CONCAT(total ORDER BY total SEPARATOR ','),
  ',', 50/100 * COUNT(*) + 1), ',', -1) AS DECIMAL) AS `Median`
FROM carts, orders
WHERE carts.total <> 0 AND carts.cart_id = orders.cart_id;


#med..n pr.ce
set @ct := (select count(1) from products);
set @row_id := 0;

select avg(truc.price) as median
from (select * from products order by products.price) as truc
where (select @row_id := @row_id + 1 as machin) AND truc.price > 0
between @ct/2.0 and @ct/2.0 + 1;

### In this database, find quartils

In [ ]:
# in orders
SELECT CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(
 GROUP_CONCAT(total ORDER BY total SEPARATOR ','),
  ',', 25/100 * COUNT(*) + 1), ',', -1) AS DECIMAL) AS `Median`
FROM carts, orders
WHERE carts.total <> 0 AND carts.cart_id = orders.cart_id;

SELECT CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(
 GROUP_CONCAT(total ORDER BY total SEPARATOR ','),
  ',', 75/100 * COUNT(*) + 1), ',', -1) AS DECIMAL) AS `Median`
FROM carts, orders
WHERE carts.total <> 0 AND carts.cart_id = orders.cart_id;


